In [6]:
import carla, time, pygame, math, random, cv2
from carla import Sensor
import math
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.mercedes.coupe_2020', rotation=None):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    if rotation:
        spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle


def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x = 2.0, z=1.0)), horizontal_fov=10, range=100, vertical_fov=10, points_per_second=1500, sensor_tick=0.0):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar_bp.set_attribute('sensor_tick', str(sensor_tick))
    radar = world.spawn_actor(radar_bp, transform=transform, attach_to=attach_to)
    return radar

def remove_all(world):
    for a in world.get_actors().filter('vehicle.*'):
        a.destroy()
    for a in world.get_actors().filter('sensor.*'):
        a.destroy()


In [7]:
from utils.detector import Detector
from utils.interpolator import Interpolator
from utils.imagewarp import ImageWarp

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [19]:
src=[[50, 240], [200, 240], [0, 0], [320, 0]]
dst=[[135, 270], [150, 270], [0, 0], [320, 0]]
scan_range={'start':0,'stop':240,'steps':10}
scan_window={'height':8,'max_adjust':8}
offset = 150

DISPLAY_COLS = 1
DISPLAY_ROWS = 2

SAVE_RESULTS = False
ESTIMATE_FROM_1_LANE = False

if ESTIMATE_FROM_1_LANE == False:
    lanes =[{'label':'mid','detections':{'start':{'x':115,'y':230},'stop':{'x':145,'y':230}}}, 
            {'label':'right','detections':{'start':{'x':145,'y':230},'stop':{'x':175,'y':230}}}
           ]
else:
    lanes =[{'label':'right','detections':{'start':{'x':145,'y':230},'stop':{'x':175,'y':230}}}
           ]

iw_obj = ImageWarp(offset=offset,src=src,dst=dst)
det_obj = Detector(scan_range=scan_range,scan_window=scan_window)
ip_obj = Interpolator(max_poly_degree=2)

a = True


In [9]:
client.load_world('Town03')

In [20]:
import manual_control
from importlib import reload
reload(manual_control)

remove_all(world)

game_loop = manual_control.setup()

vehicle = world.get_actors().filter('vehicle.*')[0]
front_camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(
        carla.Location(x=2, y=0.0, z=1.5), # specchietto retrovisore -> x=0.3, y=0.0, z=1.5),
        carla.Rotation(pitch=-10.0)
    ), width=320, height=240)

video_output = None#np.zeros((600, 800, 4), dtype=np.uint8)

def camera_callback(image):
    global a
    if a:
        global video_output
        video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
        cv2.imwrite('test.png', video_output)
        if video_output is not None:
            a = False

front_camera.listen(lambda image: camera_callback(image))

try:
    game_loop.start()
except KeyboardInterrupt:
    print('\nCancelled by user. Bye!')
    cv2.destroyAllWindows()


INFO: listening to server 127.0.0.1:2000



Welcome to CARLA manual control.

Use ARROWS or WASD keys for control.

    W            : throttle
    S            : brake
    A/D          : steer left/right
    Q            : toggle reverse
    Space        : hand-brake

    L            : toggle next light type
    Z/X          : toggle right/left blinker

    TAB          : change sensor position
    ` or N       : next sensor
    [1-9]        : change to sensor [1-9]
    C            : change weather (Shift+C reverse)

    F1           : toggle HUD
    ESC          : quit


Cancelled by user. Bye!
